In [2]:
# testing notebook
# test for query processing time
# test for different results when different methods are used.

import sys
import os
import pickle

# use abs path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from retrieval_models.data_loaders import Index
from retrieval_models.retrieval_functions import *
from retrieval_models.reranking import Reranker
from retrieval_models.evaluation_metrics import *
import tqdm 
import os
import time
from scipy.stats import kendalltau, spearmanr


In [3]:
#import all the examples
queries_examples = []
BASE_DIR = os.getcwd()
index_path = os.path.join(BASE_DIR, "data", "queries_examples.txt")
with open(index_path, 'r', encoding='utf-8', errors='replace') as file:
    queries_examples = [line.strip() for line in file if line.strip()]  # Remove empty lines

print(queries_examples[0]) #print the first one for example


TypeError: 'NoneType' object is not subscriptable in Python


In [4]:
#initialize the object index 
# ignore the errors is because of the preprocessing done at this stage and it ignores some terms
index = Index()

Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1127363:
Skipping invalid doc_info line: 	1127363:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	1127363:
Skipping invalid doc_info line: 	1449406:
Skipping invalid doc_info line: 	820167:
Skipping invalid doc_info line: 	820167:
done loading index positions
done finding vocab
c:\Users\annie\ttds_assignment\retrieval_models\data\doc_metadata.txt
done loading doc lengths


In [ ]:
# precompute the embeddings
# no need to run this again 
#print(index.vocab[1:10])
#embeddingmodel = EmbeddingModel(vocab = index.vocab)

['httpservletrequestwrapper', 'httpservletrequestwrapp', "httpservletrequest'", 'httpservletrequest', 'httpservletrequ', 'httpservletreqest', 'httpservletresponsewrapper', 'httpservletresponsewrapp', "httpservletresponse'"]


c:\Users\annie\anaconda3\anaconda\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\annie\anaconda3\anaconda\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

Precomputing embeddings...


Processing batches: 100%|██████████| 9995/9995 [5:05:03<00:00,  1.83s/it]   


✅ Embeddings successfully saved to retrieval_models/data/vocab_embeddings.pkl
🔍 File confirmed at retrieval_models/data/vocab_embeddings.pkl


In [5]:
def load_and_get_embedding(word, save_path="retrieval_models/data/vocab_embeddings.pkl"):
    with open(save_path, "rb") as f:
        embeddings, word_to_index = pickle.load(f)
    
    return embeddings[word_to_index[word]] if word in word_to_index else None

# Example usage
embedding_vector = load_and_get_embedding("java")
if embedding_vector is not None:
    print(f"Embedding: {embedding_vector[:5]} of size {len(embedding_vector)}...")  # Print first 5 values for readability


Embedding: [-0.17391707  0.34074134  0.0786799  -0.01539922  0.08461355] of size 768...


In [6]:
# Iterate over queries and print results
def run_queries(queries_examples, index, embedding_model=None, expansion=False, k=10):
    """
    Runs retrieval for a list of queries and prints top results.
    Also prints the total execution time.
    """
    start_time = time.time()  # Start timer

    retrieved_docs = {}
    
    for query in tqdm.tqdm(queries_examples):
        print("Query:", query)
        top_results = retrieval_function(query, index,embedding_model, expansion, k)
        print("Top results:", top_results)
        print("\n")
        retrieved_docs[query]=top_results

    end_time = time.time()  # End timer
    total_time = end_time - start_time

    print(f"Total Execution Time: {total_time:.4f} seconds")
    print(f"Average Time per Query: {total_time / len(queries_examples):.4f} seconds")

    return retrieved_docs


In [7]:
#the basic top 10 results for each query
retrieved_docs = run_queries(queries_examples, index)

  0%|          | 0/24 [00:00<?, ?it/s]

Query: TypeError: 'NoneType' object is not subscriptable in Python


  4%|▍         | 1/24 [00:00<00:21,  1.09it/s]

Top results: ['1408286', '1521909', '1612220', '1261875', '1274875', '1061361', '788982', '1713444', '707812', '635716']


Query: NullPointerException handling in Java


  8%|▊         | 2/24 [00:01<00:11,  1.85it/s]

...

Top results: ['1084009', '1662827', '598370', '1017046', '1103573', '1621294', '959620', '679049', '825677', '1104106']


Query: coding is very hard
Top results: ['1430292', '492284', '1104789', '492262', '1170558', '1135854', '605357', '703088', '1006463', '890929']


Query: ajdejfn code


 96%|█████████▌| 23/24 [00:07<00:00,  3.15it/s]

Top results: ['682310', '807267', '642516', '498457', '648299', '1020721', '875521', '1102436', '1088479', '1141338']


Query: how to make a website


100%|██████████| 24/24 [00:07<00:00,  3.06it/s]

Top results: ['1194880', '1512080', '1244021', '1187004', '1422187', '718924', '1337743', '1205927', '1566748', '1334971']


Total Execution Time: 7.8640 seconds
Average Time per Query: 0.3277 seconds


In [10]:
embeddingmodel = EmbeddingModel(vocab = index.vocab)
retrieved_docs_expansion = run_queries(queries_examples, index, embeddingmodel, expansion=True)

c:\Users\annie\anaconda3\anaconda\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_

Loading embeddings from retrieval_models/data/vocab_embeddings.pkl...


  0%|          | 0/24 [00:00<?, ?it/s]

Query: TypeError: 'NoneType' object is not subscriptable in Python
Word 'typeerror:' not found in precomputed embeddings. Computing on the fly?...


  0%|          | 0/24 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [7]:
#compute_clarity_score(retrieved_docs, index)

In [11]:
#testing to see whether the reranker returns them in different order

reranked = Reranker()
#print(reranked.metadata)
reranked.metadata["id"] = reranked.metadata["id"].astype(str).str.strip()
reranked.metadata.fillna(0,inplace=True)
reranked_doc_list={}
for key,retrieved_docs_list in retrieved_docs.items():
    print(retrieved_docs_list)
    print("after reranking")
    reranked_doc_list[key]=reranked.rerank(retrieved_docs_list)
    print(reranked_doc_list[key])


['1408286', '1521909', '1612220', '1261875', '1274875', '1061361', '788982', '1713444', '707812', '635716']
after reranking
['1261875', '788982', '635716', '707812', '1274875', '1061361', '1408286', '1612220', '1521909', '1713444']
['1247154', '1024206', '862483', '661131', '1223415', '1076494', '1218729', '1775881', '992937', '1040269']
after reranking
['1024206', '1247154', '1076494', '661131', '1223415', '1218729', '992937', '1040269', '862483', '1775881']
['673174', '1305042', '1003843', '1070269', '1389141', '1621999', '1350574', '1518779', '1228613', '953786']
after reranking
['673174', '1389141', '1350574', '953786', '1003843', '1305042', '1070269', '1228613', '1621999', '1518779']
['671703', '1771485', '1691014', '1770481', '925890', '1340181', '1114874', '1763334', '1637560', '1101971']
after reranking
['1771485', '671703', '1114874', '1101971', '1340181', '925890', '1763334', '1770481', '1691014', '1637560']
['1452239', '971177', '829798', '584284', '1227605', '1732030', '110

In [ ]:
for key in reranked_doc_list.items():

    kendall_tau_ = kendalltau(retrieved_docs[key], reranked_doc_list[key])
    spearman_corr_ = spearmanr(retrieved_docs[key], reranked_doc_list[key])
    print('the tau value for re ranked documents')
    print(f"Kendall's Tau: {kendall_tau_}")
    print(f"Spearman’ s Rank Correlation: {spearman_corr_}")

#If τ is low but re-ranking makes sense based on metadata/LM, it may be good.
#If τ is high, re-ranking didn’t change much.


the tau value for re ranked documents
Kendall's Tau: SignificanceResult(statistic=0.022222222222222223, pvalue=1.0)
Spearman’ s Rank Correlation: SignificanceResult(statistic=0.17575757575757575, pvalue=0.6271883447764844)
